In [1]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

In [18]:
cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path_csv = os.path.join(cwd, '..', '..', 'data')
save_path_gdb = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')
historic_path = os.path.join(cwd, 'data', 'purpleair_historic.csv')
station_path = os.path.join(cwd, 'data', 'purpleair_stations.csv')

In [42]:
# Set output point feature class name and path
output_point_fc = "stations_XYTableToPoint"
output_point_path = os.path.join(save_path_gdb, output_point_fc)

# Specify x,y fields
x_field = "longitude"
y_field = "latitude"

# Use XYTableToPoint tool to create point features
arcpy.management.XYTableToPoint(station_path, output_point_path, x_field, y_field)


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Feature Class Location: Dataset C:\Users\tande\Documents\GitHub\QualityAirQualityCities\..\..\data\QAQC.gdb does not exist or is not supported
Failed to execute (CreateFeatureclass).


ExecuteError: ERROR 000582: Error occurred during execution.


In [39]:
# Read in the CSV file with timestamps
df = pd.read_csv(historic_path)

# Convert the "timestamp" column to a pandas datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Create a list of dates to loop through
date_range = pd.date_range(start=df['timestamp'].min().date(), end=df['timestamp'].max().date())

# Loop through the dates and perform the GIS operations
for day in date_range:
    # Select the data for the current date
    query = "timestamp >= '{}' AND timestamp <= '{}'".format(day, day + pd.Timedelta(days=1))
    selection = arcpy.management.SelectLayerByAttribute("stations_XYTableToPoint", "NEW_SELECTION", query)
    
    # Perform the join and kriging operations
    arcpy.management.AddJoin(
        in_layer_or_view=selection,
        in_field="sensor_index",
        join_table=historic_path,
        join_field="sensor_index",
        join_type="KEEP_ALL",
        index_join_fields="NO_INDEX_JOIN_FIELDS"
    )

    # Perform the kriging and clip operations
    out_kriging_raster = "Kriging_{}.tif".format(day.date())
    arcpy.ddd.Kriging(
        in_point_features=selection,
        z_field="pm2_5",
        out_surface_raster=out_kriging_raster,
        semiVariogram_props="Spherical 0.000933 # # #",
        cell_size=0.000932919999999967,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    
    out_clip_raster = "Kriging_clip_{}.tif".format(day.date())
    arcpy.management.Clip(
        in_raster=out_kriging_raster,
        rectangle="-93.32910837 44.8905885090001 -93.194328522 45.0512462900001",
        out_raster=out_clip_raster,
        in_template_dataset="lab0.public.minneapolis_boundary", ##THIS IS NOT RIGHT
        nodata_value="3.4e+38",
        clipping_geometry="NONE",
        maintain_clipping_extent="NO_MAINTAIN_EXTENT"
    )

    arcpy.management.RemoveJoin(
        in_layer_or_view=selection,
        join_name=historic_path
    )


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Layer Name or Table View: Dataset stations_XYTableToPoint does not exist or is not supported
Failed to execute (SelectLayerByAttribute).
